In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from pandas_profiling import ProfileReport

D:\Users\Lenovo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [72]:
df = pd.read_csv('train_ctrUa4K.csv')

test = pd.read_csv('test_lAUu6dG.csv')

In [3]:
#profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
#profile.to_widgets()

In [4]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
df.drop('Loan_ID',axis=1,inplace=True)

In [7]:
cat_cols = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']

In [8]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [9]:
for i in cat_cols:
    print(f'Feature : {i}\nn-Unique : {df[i].nunique()}\nUnqiue : {df[i].unique()}\nValue Counts : {df[i].value_counts()}')
    print('-'*50)

Feature : Gender
n-Unique : 2
Unqiue : ['Male' 'Female' nan]
Value Counts : Male      489
Female    112
Name: Gender, dtype: int64
--------------------------------------------------
Feature : Married
n-Unique : 2
Unqiue : ['No' 'Yes' nan]
Value Counts : Yes    398
No     213
Name: Married, dtype: int64
--------------------------------------------------
Feature : Dependents
n-Unique : 4
Unqiue : ['0' '1' '2' '3+' nan]
Value Counts : 0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64
--------------------------------------------------
Feature : Education
n-Unique : 2
Unqiue : ['Graduate' 'Not Graduate']
Value Counts : Graduate        480
Not Graduate    134
Name: Education, dtype: int64
--------------------------------------------------
Feature : Self_Employed
n-Unique : 2
Unqiue : ['No' 'Yes' nan]
Value Counts : No     500
Yes     82
Name: Self_Employed, dtype: int64
--------------------------------------------------
Feature : Property_Area
n-Unique : 3
Unqiue : ['Urb

In [10]:
df.Gender = df.Gender.map({'Male':0,'Female':1})
df.Married = df.Married.map({'No':0,'Yes':1})
df.Dependents = df.Dependents.map({'0':0,'1':1,'2':2,'3+':3})
df.Education = df.Education.map({'Not Graduate':0,'Graduate':1})
df.Self_Employed = df.Self_Employed.map({'No':0,'Yes':1})
df.Loan_Status = df.Loan_Status.map({'N':0,'Y':1})

In [11]:
dummy = pd.get_dummies(df['Property_Area'],drop_first=True)
df.drop('Property_Area',axis=1,inplace=True)
df = pd.concat([df,dummy],axis=1)

In [12]:
cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Semiurban', 'Urban']

In [13]:
X = df.drop('Loan_Status',axis=1)
y = df.Loan_Status

In [14]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
X = (imputer.fit_transform(X))

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [16]:
X[0]

array([-0.48240004, -1.37154214, -0.7618924 ,  0.52836225, -0.40580562,
        0.07299082, -0.55448733, -0.09884305,  0.28002787,  0.4406627 ,
       -0.7820157 ,  1.42814704])

In [17]:
X = pd.DataFrame(X,columns=cols)

In [52]:
df_new = pd.concat([X,y],axis=1)

In [54]:
df_new.head() # used in later part

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Semiurban,Urban,Loan_Status
0,-0.4824,-1.371542,-0.761892,0.528362,-0.405806,0.072991,-0.554487,-0.098843,0.280028,0.440663,-0.782016,1.428147,1
1,-0.4824,0.730272,0.233431,0.528362,-0.405806,-0.134412,-0.038732,-0.214634,0.280028,0.440663,-0.782016,-0.700208,0
2,-0.4824,0.730272,-0.761892,0.528362,2.525543,-0.393747,-0.554487,-0.932534,0.280028,0.440663,-0.782016,1.428147,1
3,-0.4824,0.730272,-0.761892,-1.892641,-0.405806,-0.462062,0.251980,-0.307266,0.280028,0.440663,-0.782016,1.428147,1
4,-0.4824,-1.371542,-0.761892,0.528362,-0.405806,0.097728,-0.554487,-0.064106,0.280028,0.440663,-0.782016,1.428147,1


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=1)

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
##accuracy = cross_val_score(lr, X, y, scoring='accuracy', cv = 10)
##print(accuracy)
##print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [22]:
def grid_search(model,X,y,grid):
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  grid_search = GridSearchCV(estimator=model[1], param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
  grid_result = grid_search.fit(X, y)
  print('For',model[0])
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  #for mean, stdev, param in zip(means, stds, params):
      #print("%f (%f) with: %r" % (mean, stdev, param))

In [23]:
from sklearn.linear_model import LogisticRegression

model1 = ('Logistic Regression',LogisticRegression())
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid1 = dict(solver=solvers,penalty=penalty,C=c_values)

In [24]:
grid_search(model1,X,y,grid1)

For Logistic Regression
Best: 0.809942 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}


In [25]:
from sklearn.neighbors import KNeighborsClassifier

model2 = ('KNN',KNeighborsClassifier())
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
grid2 = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)

In [26]:
grid_search(model2,X,y,grid2)

For KNN
Best: 0.800185 using {'metric': 'euclidean', 'n_neighbors': 17, 'weights': 'uniform'}


In [27]:
from sklearn.svm import SVC

model3 = ('SVC',SVC())
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
grid3 = dict(kernel=kernel,C=C,gamma=gamma)

In [28]:
grid_search(model3,X,y,grid3)

For SVC
Best: 0.808311 using {'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}


In [29]:
from sklearn.ensemble import RandomForestClassifier

model4 = ('Random Forest Classifier',RandomForestClassifier())
criterion = ['gini','entropy']
max_depth = [None,3,5,7]
n_estimators = [100,200,300,400]
max_features = ['sqrt', 'log2','auto']
grid4 = dict(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth,criterion=criterion)

In [30]:
grid_search(model4,X,y,grid4)

For Random Forest Classifier
Best: 0.809942 using {'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 300}


In [47]:
from sklearn.ensemble import GradientBoostingClassifier

model5 = ('Gradient Boosting Classifier',GradientBoostingClassifier())
n_estimators = [100]
learning_rate = [0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
grid5 = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)


In [48]:
grid_search(model5,X,y,grid5)

For Gradient Boosting Classifier
Best: 0.806161 using {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}


In [33]:
lr = LogisticRegression(C=0.01,penalty='l2',solver='newton-cg')
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [34]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [35]:
print(accuracy_score(y_val,y_pred))

0.8048780487804879


In [36]:
print(confusion_matrix(y_val,y_pred))

[[16 23]
 [ 1 83]]


In [37]:
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.41      0.57        39
           1       0.78      0.99      0.87        84

    accuracy                           0.80       123
   macro avg       0.86      0.70      0.72       123
weighted avg       0.83      0.80      0.78       123



In [0]:
Best: 0.809942 using {'criterion': 'gini', 'max_depth': 3, 
                      'max_features': 'log2', 'n_estimators': 300}

In [40]:
rfc = RandomForestClassifier(criterion='gini',max_depth=3,n_estimators=300,max_features='log2')
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [41]:
print(accuracy_score(y_val,y_pred))

0.8048780487804879


In [42]:
print(confusion_matrix(y_val,y_pred))

[[16 23]
 [ 1 83]]


In [43]:
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.41      0.57        39
           1       0.78      0.99      0.87        84

    accuracy                           0.80       123
   macro avg       0.86      0.70      0.72       123
weighted avg       0.83      0.80      0.78       123



In [44]:
rfc.score(X_train,y_train)

0.814663951120163

In [46]:
rfc.score(X_val,y_val)

0.8048780487804879

In [87]:
test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Semiurban,Urban
0,0.0,1,0.0,1,0.0,5720,0,110.0,360.0,1.0,0,1
1,0.0,1,1.0,1,0.0,3076,1500,126.0,360.0,1.0,0,1
2,0.0,1,2.0,1,0.0,5000,1800,208.0,360.0,1.0,0,1
3,0.0,1,2.0,1,0.0,2340,2546,100.0,360.0,NaN,0,1
4,0.0,0,0.0,0,0.0,3276,0,78.0,360.0,1.0,0,1


In [74]:
loan_id = test.iloc[:,0]

In [75]:
test.drop('Loan_ID',axis=1,inplace=True)

In [76]:
test.Gender = test.Gender.map({'Male':0,'Female':1})
test.Married = test.Married.map({'No':0,'Yes':1})
test.Dependents = test.Dependents.map({'0':0,'1':1,'2':2,'3+':3})
test.Education = test.Education.map({'Not Graduate':0,'Graduate':1})
test.Self_Employed = test.Self_Employed.map({'No':0,'Yes':1})

In [77]:
dummy = pd.get_dummies(test['Property_Area'],drop_first=True)
test.drop('Property_Area',axis=1,inplace=True)
test = pd.concat([test,dummy],axis=1)

In [78]:
cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Semiurban', 'Urban']

In [81]:
X_test = test

In [82]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
X_test = (imputer.fit_transform(X_test))

In [83]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test = sc.fit_transform(X_test)

In [84]:
X_test = pd.DataFrame(X_test,columns=cols)

In [86]:
X_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Semiurban,Urban
0,-0.497424,0.758358,-0.784538,0.544812,-0.361599,0.186461,-0.673335,-0.436564,0.270817,0.458850,-0.679817,1.273353
1,-0.497424,0.758358,0.150285,0.544812,-0.361599,-0.352692,-0.029848,-0.177100,0.270817,0.458850,-0.679817,1.273353
2,-0.497424,0.758358,1.085108,0.544812,-0.361599,0.039641,0.098849,1.152652,0.270817,0.458850,-0.679817,1.273353
3,-0.497424,0.758358,1.085108,0.544812,-0.361599,-0.502774,0.418877,-0.598728,0.270817,-0.446515,-0.679817,1.273353
4,-0.497424,-1.318638,-0.784538,-1.835497,-0.361599,-0.311909,-0.673335,-0.955491,0.270817,0.458850,-0.679817,1.273353


In [69]:
# using logistic Regression for prediction

In [88]:
lr.fit(X_train,y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
prediction = lr.predict(X_test)

In [92]:
submission = pd.concat([loan_id,pd.Series(prediction)],axis=1)

In [95]:
submission = submission.rename(columns={0:'Loan_Status'})

In [97]:
submission['Loan_Status'] = submission['Loan_Status'].map({0:'N',1:'Y'})

In [101]:
submission = submission.set_index('Loan_ID')

In [102]:
submission.to_csv('Submission_01.csv')